# Neurocombat TOP and StrokeMRI harmonized datasets

Note this must be run in the `neurocombaty` environment

## import libraries

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from neuroCombat import neuroCombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
filepath = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath,'StrokeMRI_pvc2c.csv') 
filename_top = os.path.join(filepath,'TOP_pvc2c.csv') 

In [ ]:
our_mri_data = pd.read_csv(filename_mri)
our_top_data = pd.read_csv(filename_top)

In [ ]:
print("Our StrokeMRI data is", len(our_mri_data), "patients")
print("Our TOP data is", len(our_top_data), "patients")

In [ ]:
our_top_data.head(3)

In [ ]:
our_mri_data.head(3)

In [ ]:
top_selection = our_top_data.drop(['Unnamed: 0'],axis=1)
mri_selection = our_mri_data.drop(['Unnamed: 0'],axis=1)

In [ ]:
#mri_selection.isna().sum()
#top_selection.isna().sum()

In [ ]:
mri_selection = mri_selection.set_index('participant_id')
top_selection = top_selection.set_index('participant_id')

In [ ]:
top_selection = top_selection.T
mri_selection = mri_selection.T
#mri_selection

In [ ]:
both_together = pd.concat([ mri_selection, top_selection], axis=1, join="inner")
both_together

In [ ]:
# make sure there are no NaNs
both_together.isna().sum().sum()

In [ ]:
# # save off csv
# both_together.to_csv('both_top_mri_together.csv')

In [ ]:
# make and save of csv of features only
features_only = both_together[2:]
features_only.to_csv('features_only_top_mri.csv')

In [ ]:
features_only.T.columns

In [ ]:
dictionary_features_len = len(features_only.T.columns)
number = 0
made_keys = [] 
made_vals = [] 
for n in features_only.T.columns:
    
    made_keys.append(number)
    made_vals.append(n)
    number +=1

In [ ]:
feature_dict = dict(map(lambda i,j : (i,j) , made_keys,made_vals))

In [ ]:
ft = features_only.reset_index()
ft = ft.rename(columns={"index": "A"})
ft = ft.drop(['A'], axis=1)
ft = ft.dropna()

In [ ]:
bt= both_together.reset_index()
bt = bt.rename(columns={"index": "A"})
bt = bt.drop(['A'], axis=1)
bt = bt.dropna()

In [ ]:
bt.head(3)

In [ ]:
ft.to_csv('ft_top_mri.csv')

In [ ]:
data = np.genfromtxt('ft_top_mri.csv', delimiter=",", skip_header=1)
data

In [ ]:
data = data[:, 1:]
#data

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len(mri_selection.columns)
last_columns_as_two = [2] * len(top_selection.columns)
covars = {'batch':first_columns_as_one + last_columns_as_two,
          'sex':both_together.loc['sex',:].values.tolist(),
           'age':both_together.loc['age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
#covars

In [ ]:
covars.shape

In [ ]:
data.shape

In [ ]:
# specifify sex as categorical
categorical_cols = ['sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'

#Harmonization step:
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    categorical_cols=categorical_cols)["data"]

In [ ]:
data_combat

In [ ]:
data_combat.shape

In [ ]:
neurocombat = pd.DataFrame(data_combat)
neurocombat

In [ ]:
topper = bt.head(2)

In [ ]:
topper = topper.rename_axis(None, axis="columns")

In [ ]:
topper

In [ ]:
topper = topper.reset_index(drop=False)
topper = topper.rename(columns={"index": "char"})

In [ ]:
topper['char'][0] = 'age'
topper['char'][1] = 'sex'

In [ ]:
topper

In [ ]:
bottom = neurocombat.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topper.columns
#bottom = bottom.set_index('char', inplace=True)
#set_index('Name', inplace=True)
bottom

In [ ]:
back_together = pd.concat([topper, bottom])
back_together = back_together.T
back_together

In [ ]:


new_header = back_together.iloc[0] #grab the first row for the header
#back_together = back_together[1:] #take the data less the header row
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
back_together


In [ ]:
back_together.head(514).tail(10)

In [ ]:
back_together.tail(527).head(10)

In [ ]:
neuro_harm_top =back_together.tail(527)
neuro_harm_mri =back_together.head(514)

In [ ]:
neuro_harm_top = neuro_harm_top.rename(feature_dict, axis='columns')
neuro_harm_mri = neuro_harm_mri.rename(feature_dict, axis='columns')

In [ ]:
neuro_harm_mri.to_csv('neuro_harm_mri.csv')
neuro_harm_top.to_csv('neuro_harm_top.csv')